Linear Elasticity
===

In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

importing NGSolve-6.2.2204


A rectangular mesh:

In [2]:
def GenerateMesh(t=0.1):
    bar = MoveTo(0,-t/2).Rectangle(1,t).Face()
    bar.edges.Min(X).name="left"
    bar.edges.Max(X).name="right"
    bar.edges.Min(X).maxh=t/6
    mesh = Mesh(OCCGeometry(bar, dim=2).GenerateMesh(maxh=t/3))
    return mesh
mesh = GenerateMesh()
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

In [3]:
force = CF( (0,0.1) )

E = 100    # Young modulus 
nu = 0.2   # Poisson ration (Querkontraktionszahl)
# Lame parameters
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Hooke(eps):
    return 2*mu*eps + lam*Trace(eps)*Id(2)

$$ \int_{\Omega}\mathbb{C}\varepsilon(u):\varepsilon(v)\,dx = \int_{\Omega}f\cdot v\,dx + \int_{\Gamma_N} g\cdot v\,ds $$

In [4]:
def Solve(mesh, order=2):
    fes = VectorH1(mesh, order=order, dirichlet="right")
    u,v = fes.TnT()

    a = BilinearForm(fes, symmetric=True)
    a += InnerProduct(Hooke(Sym(Grad(u))),Sym(Grad(v)))*dx
    #a += 2*mu*InnerProduct(Sym(Grad(u)),Sym(Grad(v)))+lam*div(u)*div(v)*dx

    f = LinearForm(fes)
    f += force*v*ds("left") # shear force at right boundary
    # f += force*v*dx

    gfu = GridFunction(fes)
    a.Assemble()
    f.Assemble()

    gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    return gfu

In [5]:
mesh = GenerateMesh()
gfu = Solve(mesh)
Draw (Sym(Grad(gfu))[0,0], mesh, deformation=gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene